In [2]:
#Perceptual hash algorithm+ASOCT_cataract
#each image --> fingerprint
import sys
import os
import math
import random
import heapq 
import numpy as np
import pandas as pd
from PIL import Image
import psycopg2 as ps
from io import StringIO,BytesIO 
import pywt
import cv2
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from functools import reduce

In [ ]:
#1. create table
conn = ps.connect(host="127.0.0.1", user="postgres", password="postgres", database="asoct")#connect postgresql 
if conn is not None:
    cur = conn.cursor()
    command = "create table cataract_ph (name text NOT NULL, cA_img float8[] NOT NULL, signature BIGINT NOT NULL);"
    cur.execute(command)
    conn.commit()# commit the changes
    cur.close()# close communication with the PostgreSQL database server
conn.close()

In [17]:
#2. Extract features with wavelet transform and get fingerprint
def avhash(im): #Perceptual hash algorithm
    if not isinstance(im, Image.Image):
        im = Image.open(im)
    im = im.resize((7, 7), Image.ANTIALIAS).convert('L')
    avg = reduce(lambda x, y: x + y, im.getdata()) / 64.
    return reduce(lambda x, yz: x | (yz[1] << yz[0]),
                  enumerate(map(lambda i: 0 if i < avg else 1, im.getdata())),
                  0)
#insert into database
conn = ps.connect(host="127.0.0.1", user="postgres", password="postgres", database="asoct")#connect postgresql 
if conn is not None:
    cur = conn.cursor()
    image_dir = '/data/fjsdata/ASOCT/Cataract/C_8bit_Crop_New' #the path of images
    num_img = 0 #count
    for fname in sorted(os.listdir(image_dir)):
        if fname.endswith(".jpg"):
            image_path = os.path.join(image_dir, fname)
            try:
                img = cv2.imread(image_path)
                if img is not None:
                    img = cv2.resize(img, (1000, 200))#resize image
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32)  #turn  multi-channel to single-channel
                    cA, (cH, cV, cD) = pywt.dwt2(data=img, wavelet='haar')#Single level Discrete Wavelet Transform.
                    command = "insert into cataract_ph(name, cA_img, signature) values(%s, %s, %s);"
                    params = (fname, cA.flatten().tolist(), avhash(image_path))
                    cur.execute(command, params)
                    conn.commit()# commit the changes  
            except:
                print(fname+":"+str(avhash(image_path)))
        num_img = num_img+1
        sys.stdout.write('\r{} / {} '.format(num_img,40066))
        sys.stdout.flush()
    cur.close()
conn.close()

40066 / 40066 

In [4]:
#performance metric
data = pd.read_csv("/data/fjsdata/ASOCT/Cataract/CBIR_Cataract.csv" , sep=',')#load dataset
print('Dataset Statistics: Rows = %d, Cols = %d' % (data.shape[0], data.shape[1]))
#labels_dis = data[['D']].drop_duplicates()#observe the distribution of labels
#load images
name_list = data['A'].tolist()
def hit(name, smname):
    if smname not in name_list:#no label
        return 0
    else:
        lv_name = data.loc[data.A==name,'D'].values[0]
        lv_smname = data.loc[data.A==smname,'D'].values[0]
        if ( lv_name== lv_smname): #same level,hit
            return 1
        return 0  
    
conn = ps.connect(host="127.0.0.1", user="postgres", password="postgres", database="asoct")#connect postgresql 
if conn is not None:
    cur = conn.cursor() 
    for topk in [5,10,15]:
        HR =[] #Hit ratio 
        MAP =[] #mean average precision
        for name in random.sample(name_list,1000):
            #command = "SELECT name, smlr FROM (SELECT name, smlar(cA_img,(SELECT cA_img FROM cataract_ph WHERE name = %s)) AS smlr FROM cataract_ph WHERE name <> %s ORDER BY (signature::bit(64) & (SELECT signature FROM cataract_ph WHERE name = %s)::bit(64)) LIMIT 100) x ORDER BY x.smlr ASC LIMIT %s;"
            #command = "SELECT name, smlr FROM (SELECT name, smlar(cA_img,(SELECT cA_img FROM cataract_ph WHERE name = %s)) AS smlr FROM cataract_ph WHERE name in (SELECT name FROM pat ORDER BY signature <-> (SELECT signature FROM pat WHERE name = %s) LIMIT 100) ) x ORDER BY x.smlr ASC LIMIT %s;"
            command = "SELECT name, smlr FROM (SELECT name, (signature::bit(64) & (SELECT signature FROM cataract_ph WHERE name = %s)::bit(64)) AS smlr FROM cataract_ph WHERE name in (SELECT name FROM pat ORDER BY signature <-> (SELECT signature FROM pat WHERE name = %s) LIMIT 100) ) x ORDER BY x.smlr ASC LIMIT %s;"
            params = (name, name, topk)
            cur.execute(command, params)
            #perfromance
            pos_len = 0
            rank_len = 0
            rank_map = []
            for smname, smlr in cur.fetchall():
                ret = hit(name,smname)
                HR.append(ret)
                rank_len=rank_len+1
                if ret==1: 
                    pos_len = pos_len +1
                    rank_map.append(pos_len/rank_len) 
                else: 
                    rank_map.append(0)                  
            MAP.append(np.mean(rank_map))  
        print("HR@{}={:.6f}, MAP@{}={:.6f}".format(topk,np.mean(HR),topk,np.mean(MAP)))              
    cur.close()
conn.close()

Dataset Statistics: Rows = 31998, Cols = 4
HR@5=0.263200, MAP@5=0.191700
HR@10=0.275300, MAP@10=0.179515
HR@15=0.290533, MAP@15=0.170046


In [8]:
#!/usr/bin/python
import glob
import os
import sys
from functools import reduce
from PIL import Image

EXTS = 'jpg', 'jpeg', 'JPG', 'JPEG', 'gif', 'GIF', 'png', 'PNG'

def avhash(im): #Perceptual hash algorithm
    if not isinstance(im, Image.Image):
        im = Image.open(im)
    im = im.resize((8, 8), Image.ANTIALIAS).convert('L')
    avg = reduce(lambda x, y: x + y, im.getdata()) / 64.
    return reduce(lambda x, yz: x | (yz[1] << yz[0]),
                  enumerate(map(lambda i: 0 if i < avg else 1, im.getdata())),
                  0)

def hamming(h1, h2):
    h, d = 0, h1 ^ h2
    while d:
        h += 1
        d &= d - 1
    return h

img1 = avhash('/data/fjsdata/ASOCT/Cataract/C_8bit_Crop_New/c0100_20181102_111708_R_CASIA2_LGC_002.jpg')
print(img1)
img2 = avhash('/data/fjsdata/ASOCT/Cataract/C_8bit_Crop_New/C020_20180514_100425_L_CASIA2_LGC_008.jpg')
print(img2)
print (hamming(img1, img2))
'''
if __name__ == '__main__':
    if len(sys.argv) <= 1 or len(sys.argv) > 3:
        print "Usage: %s image.jpg [dir]" % sys.argv[0]
    else:
        im, wd = sys.argv[1], '.' if len(sys.argv) < 3 else sys.argv[2]
        h = avhash(im)

        os.chdir(wd)
        images = []
        for ext in EXTS:
            images.extend(glob.glob('*.%s' % ext))

        seq = []
        prog = int(len(images) > 50 and sys.stdout.isatty())
        for f in images:
            seq.append((f, hamming(avhash(f), h)))
            if prog:
                perc = 100. * prog / len(images)
                x = int(2 * perc / 5)
                print '\rCalculating... [' + '#' * x + ' ' * (40 - x) + ']',
                print '%.2f%%' % perc, '(%d/%d)' % (prog, len(images)),
                sys.stdout.flush()
                prog += 1

        if prog: print
        for f, ham in sorted(seq, key=lambda i: i[1]):
            print "%d\t%s" % (ham, f)
'''

1188703169215789056
35678052826480384
1153447328871548672
18


'\nif __name__ == \'__main__\':\n    if len(sys.argv) <= 1 or len(sys.argv) > 3:\n        print "Usage: %s image.jpg [dir]" % sys.argv[0]\n    else:\n        im, wd = sys.argv[1], \'.\' if len(sys.argv) < 3 else sys.argv[2]\n        h = avhash(im)\n\n        os.chdir(wd)\n        images = []\n        for ext in EXTS:\n            images.extend(glob.glob(\'*.%s\' % ext))\n\n        seq = []\n        prog = int(len(images) > 50 and sys.stdout.isatty())\n        for f in images:\n            seq.append((f, hamming(avhash(f), h)))\n            if prog:\n                perc = 100. * prog / len(images)\n                x = int(2 * perc / 5)\n                print \'\rCalculating... [\' + \'#\' * x + \' \' * (40 - x) + \']\',\n                print \'%.2f%%\' % perc, \'(%d/%d)\' % (prog, len(images)),\n                sys.stdout.flush()\n                prog += 1\n\n        if prog: print\n        for f, ham in sorted(seq, key=lambda i: i[1]):\n            print "%d\t%s" % (ham, f)\n'